In [11]:
import gurobipy as gp
from gurobipy import GRB, quicksum

# ===================================================
# 1) DATA
# ===================================================

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

# Which products are woven vs. knit
product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

# Grams of raw material needed per unit
raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

# Minimum units required
min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw material capacity: 15 kg for woven, 23 kg for knit (in grams)
RM_WOVEN_MAX = 15000
RM_KNIT_MAX  = 23000

# ---------- COSTS -----------
#  A) Raw material cost (INR/g)
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25

#  B) Spinning cost per gram (same for all products in the original table)
spin_labour_cost = 3.00

#  C) Weaving cost per gram (for the woven SKUs, else 0)
weave_labour_cost = {
    "Shawl":   1.46,
    "Stole":   1.79,
    "Scarf":   3.00,
    "Sweater": 0.0,
    "Muffler": 0.0,
    "Cap":     0.0,
    "Gloves":  0.0
}

#  D) Knitting cost per gram (for the knit SKUs, else 0)
knit_labour_cost = {
    "Shawl":   0.0,
    "Stole":   0.0,
    "Scarf":   0.0,
    "Sweater": 3.00,
    "Muffler": 3.00,
    "Cap":     3.00,
    "Gloves":  3.00
}

#  E) Finishing cost per gram (applies to all products)
finishing_labour_cost = 0.60

#  F) Dye cost per product (fixed cost per dyed unit)
dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

#  G) Profit margin
#     Price = Cost * (1 + margin)
#     => Profit per unit = Cost * margin
profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

# ---------- CAPACITY (Knitting/Weaving/Finishing) -----------
# Number of knitters, weavers, and days
knitter_count = 13
weaver_count  = 5
finisher_count = 0  # If you have finishing constraints, put the # of finishers here

D = 5  # days for production

# Productivity (grams/day/person)
weave_prod_g_per_day = 500
knit_prod_g_per_day  = 50

# If you also want finishing capacity, define finishing_prod_g_per_day
# fin_prod_g_per_day = ????


# ===================================================
# 2) HELPER FUNCTIONS
# ===================================================

def unit_cost(product, dyed=False):
    """
    Return the *full* cost of producing one unit of `product`,
    including raw material, spinning, weaving/knitting, finishing, and
    the *fixed dye cost* if dyed.
    """
    grams = raw_req[product]
    
    # (1) Raw material cost
    if product_type[product] == "woven":
        mat_c = grams * raw_material_cost_woven
    else:
        mat_c = grams * raw_material_cost_knit
    
    # (2) Spinning cost
    spin_c   = grams * spin_labour_cost  # from the table, 3.00 INR/g for all SKUs
    
    # (3) Weaving or knitting cost
    wv_c = grams * weave_labour_cost[product]
    kn_c = grams * knit_labour_cost[product]
    
    # (4) Finishing cost
    finish_c = grams * finishing_labour_cost
    
    # (5) Dye cost (only if dyed)
    dye_c = dye_cost[product] if dyed else 0.0
    
    return mat_c + spin_c + wv_c + kn_c + finish_c + dye_c


def unit_profit(product, dyed=False):
    """
    Profit per unit = cost * margin (since Price = cost * (1 + margin)).
    """
    c = unit_cost(product, dyed)
    return c * profit_margin[product]


def unit_selling_price(product, dyed=False):
    """
    Selling price = cost * (1 + margin).
    """
    c = unit_cost(product, dyed)
    m = profit_margin[product]
    return c * (1 + m)


# ===================================================
# 3) BUILD MODEL
# ===================================================

m = gp.Model("FullCost_IncludingSpinningEtc")

# Decision variables: xD[i], xN[i] => # of dyed vs. natural units for each SKU
xD = {}
xN = {}
for p in products:
    xD[p] = m.addVar(vtype=GRB.INTEGER, name=f"xDyed_{p}", lb=0)
    xN[p] = m.addVar(vtype=GRB.INTEGER, name=f"xNat_{p}",  lb=0)

# Objective: max total profit = sum of (profit/unit * units)
m.setObjective(
    quicksum(
        unit_profit(p, True)*xD[p] + unit_profit(p, False)*xN[p]
        for p in products
    ),
    GRB.MAXIMIZE
)

# ===================================================
# 4) CONSTRAINTS
# ===================================================

# (a) Minimum units required
for p in products:
    m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")

# (b) <= 60% dyed
for p in products:
    m.addConstr(
        xD[p] == 0.60*(xD[p] + xN[p]),
        name=f"Max60percentDyed_{p}"
    )

# (c) Raw material capacity: woven vs. knit
m.addConstr(
    quicksum(raw_req[p]*(xD[p] + xN[p]) for p in products if product_type[p] == "woven")
    <= RM_WOVEN_MAX,
    name="WovenRawLimit"
)
m.addConstr(
    quicksum(raw_req[p]*(xD[p] + xN[p]) for p in products if product_type[p] == "knit")
    <= RM_KNIT_MAX,
    name="KnitRawLimit"
)

# (d) Weaving capacity
#  grams needed for all woven SKUs <= (weavers * weave_prod_g_per_day * D)
weave_capacity = weaver_count * weave_prod_g_per_day * D
m.addConstr(
    quicksum(raw_req[p]*(xD[p] + xN[p]) for p in products if product_type[p] == "woven")
    <= weave_capacity,
    name="WeaveCapacity"
)

# (e) Knitting capacity
knit_capacity = knitter_count * knit_prod_g_per_day * D
m.addConstr(
    quicksum(raw_req[p]*(xD[p] + xN[p]) for p in products if product_type[p] == "knit")
    <= knit_capacity,
    name="KnitCapacity"
)

# (OPTIONAL) If finishing is also a bottleneck, you can do:
# finish_capacity = finisher_count * fin_prod_g_per_day * D
# m.addConstr(
#   quicksum(raw_req[p]*(xD[p] + xN[p]) for p in products)
#   <= finish_capacity,
#   "FinishCap"
# )

# ===================================================
# 5) OPTIMIZE
# ===================================================
m.optimize()

# ===================================================
# 6) RESULTS
# ===================================================
if (m.Status == GRB.INFEASIBLE) or (m.SolCount == 0):
    print("\nNO FEASIBLE SOLUTION.\n")
else:
    print(f"\nOptimal Total Profit = {m.ObjVal:.2f} INR\n")
    # For each product, show how many dyed vs. natural,
    # plus the cost, selling price, and profit per unit for each mode.
    for p in products:
        xDval = xD[p].X
        xNval = xN[p].X
        cost_dyed   = unit_cost(p, True)
        cost_natural= unit_cost(p, False)
        profit_dyed = unit_profit(p, True)
        profit_nat  = unit_profit(p, False)
        price_dyed  = unit_selling_price(p, True)
        price_nat   = unit_selling_price(p, False)
        
        print(f"{p:8s} => Dyed={xDval:7.2f}, Natural={xNval:7.2f}")
        print(f"      Per-unit cost(dyed)={cost_dyed:8.2f}, "
              f"sell={price_dyed:8.2f}, profit={profit_dyed:8.2f}")
        print(f"      Per-unit cost(nat )={cost_natural:8.2f}, "
              f"sell={price_nat:8.2f}, profit={profit_nat:8.2f}")
        print("------------------------------------------------")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 14 columns and 56 nonzeros
Model fingerprint: 0xf985b95b
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-01, 5e+02]
  Objective range  [2e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 18 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 65689.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.568916250000e+04, best bound 6.568916250000e+04, gap 0.0000%

Optimal Total Profit = 65689.16 INR

Shawl    => Dyed=   6.00, Natural=   4.00
  

In [16]:
import gurobipy as gp
from gurobipy import GRB, quicksum

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw-material capacity (grams)
RM_WOVEN_MAX = 15000
RM_KNIT_MAX  = 23000

# COST PARAMETERS
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25
spin_labour_cost        = 3.00

weave_labour_cost = {
    "Shawl":   1.46,
    "Stole":   1.79,
    "Scarf":   3.00,
    "Sweater": 0.0,
    "Muffler": 0.0,
    "Cap":     0.0,
    "Gloves":  0.0
}

knit_labour_cost = {
    "Shawl":   0.0,
    "Stole":   0.0,
    "Scarf":   0.0,
    "Sweater": 3.00,
    "Muffler": 3.00,
    "Cap":     3.00,
    "Gloves":  3.00
}

finishing_labour_cost = 0.60

dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

# Workforce
knitter_count = 13
weaver_count  = 5

weave_prod_g_per_day = 500
knit_prod_g_per_day  = 50

def unit_cost(product, dyed=False):
    """Full cost including raw, spinning, weave/knit, finishing, dye."""
    grams = raw_req[product]
    # raw material
    mat_c = grams * (raw_material_cost_woven if product_type[product]=="woven"
                     else raw_material_cost_knit)
    # spinning
    spin_c = grams * spin_labour_cost
    # weaving or knitting
    wv_c   = grams * weave_labour_cost[product]
    kn_c   = grams * knit_labour_cost[product]
    # finishing
    fin_c  = grams * finishing_labour_cost
    # dye
    dye_c  = dye_cost[product] if dyed else 0.0
    return mat_c + spin_c + wv_c + kn_c + fin_c + dye_c

def unit_profit(product, dyed=False):
    c = unit_cost(product, dyed)
    return c * profit_margin[product]

def unit_selling_price(product, dyed=False):
    c = unit_cost(product, dyed)
    return c*(1 + profit_margin[product])

# Now we vary D from 3 to 32, step=2 => 3,5,7,...,31
for D in range(3, 50, 1):
    m = gp.Model(f"VaryDays_D{D}")
    m.setParam("OutputFlag", 0)

    xD = {}
    xN = {}
    for p in products:
        xD[p] = m.addVar(vtype=GRB.INTEGER, name=f"xDyed_{p}")
        xN[p] = m.addVar(vtype=GRB.INTEGER, name=f"xNat_{p}")

    # Objective
    m.setObjective(
        quicksum(unit_profit(p, True)*xD[p] + unit_profit(p, False)*xN[p]
                 for p in products),
        GRB.MAXIMIZE
    )

    # Constraints
    for p in products:
        m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")
        m.addConstr(xD[p] == 0.60*(xD[p] + xN[p]), name=f"DyeLimit_{p}")

    # raw material capacity
    m.addConstr(quicksum(raw_req[p]*(xD[p]+xN[p]) 
                         for p in products if product_type[p]=="woven")
                <= RM_WOVEN_MAX, "WovenRawCap")

    m.addConstr(quicksum(raw_req[p]*(xD[p]+xN[p]) 
                         for p in products if product_type[p]=="knit")
                <= RM_KNIT_MAX,  "KnitRawCap")

    # weaving capacity
    weave_capacity = weaver_count * weave_prod_g_per_day * D
    m.addConstr(quicksum(raw_req[p]*(xD[p]+xN[p]) 
                         for p in products if product_type[p]=="woven")
                <= weave_capacity, "WeaveCap")

    # knitting capacity
    knit_capacity = knitter_count * knit_prod_g_per_day * D
    m.addConstr(quicksum(raw_req[p]*(xD[p]+xN[p]) 
                         for p in products if product_type[p]=="knit")
                <= knit_capacity, "KnitCap")

    m.optimize()

    print(f"=== D={D} days ===")
    if m.Status == GRB.INFEASIBLE or m.SolCount == 0:
        print("NO SOLUTION\n")
    else:
        print(f"Optimal Profit = {m.ObjVal:.2f}")
        for p in products:
            xdv = xD[p].X
            xnv = xN[p].X
            if xdv + xnv > 1e-6:  # only print if >0
                c_d  = unit_cost(p, True)
                c_n  = unit_cost(p, False)
                prof_d = unit_profit(p, True)
                prof_n = unit_profit(p, False)
                sp_d   = unit_selling_price(p, True)
                sp_n   = unit_selling_price(p, False)
                print(f"  {p:8s}: Dyed={xdv:6.2f}, Nat={xnv:6.2f}")
                print(f"     costDye={c_d:7.2f}, profitDye={prof_d:7.2f}, sellDye={sp_d:7.2f}")
                print(f"     costNat={c_n:7.2f}, profitNat={prof_n:7.2f}, sellNat={sp_n:7.2f}")
        print("------------------------------------------------\n")


=== D=3 days ===
NO SOLUTION

=== D=4 days ===
NO SOLUTION

=== D=5 days ===
Optimal Profit = 65689.16
  Shawl   : Dyed=  6.00, Nat=  4.00
     costDye=8638.10, profitDye=2159.53, sellDye=10797.62
     costNat=7808.10, profitNat=1952.03, sellNat=9760.12
  Stole   : Dyed=  3.00, Nat=  2.00
     costDye=6734.60, profitDye=1683.65, sellDye=8418.25
     costNat=6099.60, profitNat=1524.90, sellNat=7624.50
  Scarf   : Dyed= 15.00, Nat= 10.00
     costDye=3880.50, profitDye= 970.12, sellDye=4850.62
     costNat=3538.50, profitNat= 884.62, sellNat=4423.12
  Sweater : Dyed=  3.00, Nat=  2.00
     costDye=5880.00, profitDye=1470.00, sellDye=7350.00
     costNat=5392.00, profitNat=1348.00, sellNat=6740.00
  Muffler : Dyed=  3.00, Nat=  2.00
     costDye=3233.75, profitDye= 808.44, sellDye=4042.19
     costNat=2948.75, profitNat= 737.19, sellNat=3685.94
  Cap     : Dyed=  3.00, Nat=  2.00
     costDye=1217.25, profitDye= 243.45, sellDye=1460.70
     costNat=1095.25, profitNat= 219.05, sellNat=1314

In [24]:
import gurobipy as gp
from gurobipy import GRB, quicksum

# =========================================
# 1) Base Data
# =========================================

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw-material capacity (grams)
RM_WOVEN_MAX = 15000  # 15 kg for woven
RM_KNIT_MAX  = 23000  # 23 kg for knit

# Base profit margin (we will add increments to these)
base_profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

# Cost parameters
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25
spin_labour_cost        = 3.00

weave_labour_cost = {
    "Shawl":   1.46,
    "Stole":   1.79,
    "Scarf":   3.00,
    "Sweater": 0.00,
    "Muffler": 0.00,
    "Cap":     0.00,
    "Gloves":  0.00
}

knit_labour_cost = {
    "Shawl":   0.00,
    "Stole":   0.00,
    "Scarf":   0.00,
    "Sweater": 3.00,
    "Muffler": 3.00,
    "Cap":     3.00,
    "Gloves":  3.00
}

finishing_labour_cost = 0.60

dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

# Weavers / knitters
weaver_count  = 5
knitter_count = 13

# Productivity in grams/person/day
weave_prod_g_per_day = 500
knit_prod_g_per_day  = 50

# =========================================
# 2) Cost / Profit Functions
# =========================================

def compute_unit_cost(product, dyed=False):
    """
    Return full cost of producing one unit, including raw, spinning, weaving/knit, finishing, dye if dyed.
    """
    grams = raw_req[product]
    
    # raw material
    if product_type[product] == "woven":
        mat_c = grams * raw_material_cost_woven
    else:
        mat_c = grams * raw_material_cost_knit
    
    # spinning
    spin_c = grams * spin_labour_cost
    
    # weaving or knitting
    wv_c   = grams * weave_labour_cost[product]
    kn_c   = grams * knit_labour_cost[product]
    
    # finishing
    fin_c  = grams * finishing_labour_cost
    
    # dye
    dye_c  = dye_cost[product] if dyed else 0.0
    
    return mat_c + spin_c + wv_c + kn_c + fin_c + dye_c

def compute_unit_profit(product, dyed, margin_dict):
    """
    Profit per unit = cost * margin
    (since Price = cost * (1 + margin)).
    """
    c = compute_unit_cost(product, dyed)
    return c * margin_dict[product]

# =========================================
# 3) Loops Over margin_increments & Days
# =========================================

margin_increments = [0,0.05, 0.10, 0.15, 0.20]
days_range = range(3, 40, 1)  # from 3 to 39 inclusive

for inc in margin_increments:
    # Build scenario margin dictionary
    scenario_margin = {}
    for p in products:
        scenario_margin[p] = base_profit_margin[p] + inc
    
    for D in days_range:
        scenario_label = f"Margin+{int(inc*100)}%_Days={D}"
        
        # Build model
        m = gp.Model(scenario_label)
        m.setParam("OutputFlag", 0)  # quiet solver output
        
        # Decision vars
        xD = {}
        xN = {}
        for p in products:
            xD[p] = m.addVar(vtype=GRB.INTEGER, name=f"xDyed_{p}")
            xN[p] = m.addVar(vtype=GRB.INTEGER, name=f"xNat_{p}")

        # Objective
        m.setObjective(
            quicksum(
                compute_unit_profit(p, True,  scenario_margin)*xD[p] +
                compute_unit_profit(p, False, scenario_margin)*xN[p]
                for p in products
            ),
            GRB.MAXIMIZE
        )

        # Constraints
        
        # (a) Minimum units
        for p in products:
            m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")

        # (b) <= 60% dyed
        for p in products:
            m.addConstr(xD[p] == 0.60*(xD[p] + xN[p]), name=f"DyeLimit_{p}")

        # (c) raw material capacity
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
            <= RM_WOVEN_MAX,
            name="WovenRawCap"
        )
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
            <= RM_KNIT_MAX,
            name="KnitRawCap"
        )

        # (d) Weaving capacity
        weave_cap = weaver_count*weave_prod_g_per_day*D
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
            <= weave_cap,
            "WeaveCap"
        )

        # (e) Knitting capacity
        knit_cap = knitter_count*knit_prod_g_per_day*D
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
            <= knit_cap,
            "KnitCap"
        )

        # Solve
        m.optimize()

        # Print results
        print(f"=== SCENARIO: {scenario_label} ===")
        if m.Status == GRB.INFEASIBLE or m.SolCount == 0:
            print("   NO SOLUTION\n")
            continue
        
        print(f"   Optimal Total Profit = {m.ObjVal:.2f}")
        for p in products:
            xd_val = xD[p].X
            xn_val = xN[p].X
            if (xd_val + xn_val) > 1e-6:  # only print details if > 0
                # cost
                cost_d = compute_unit_cost(p, True)
                cost_n = compute_unit_cost(p, False)
                # profit
                prof_d = compute_unit_profit(p, True,  scenario_margin)
                prof_n = compute_unit_profit(p, False, scenario_margin)
                print(f"   {p:8s}: Dyed={xd_val:6.2f}, Nat={xn_val:6.2f}, "
                      f"costDye={cost_d:7.2f}, profitDye={prof_d:7.2f}, "
                      f"costNat={cost_n:7.2f}, profitNat={prof_n:7.2f}")
        print("------------------------------------------------\n")


=== SCENARIO: Margin+0%_Days=3 ===
   NO SOLUTION

=== SCENARIO: Margin+0%_Days=4 ===
   NO SOLUTION

=== SCENARIO: Margin+0%_Days=5 ===
   Optimal Total Profit = 65689.16
   Shawl   : Dyed=  6.00, Nat=  4.00, costDye=8638.10, profitDye=2159.53, costNat=7808.10, profitNat=1952.03
   Stole   : Dyed=  3.00, Nat=  2.00, costDye=6734.60, profitDye=1683.65, costNat=6099.60, profitNat=1524.90
   Scarf   : Dyed= 15.00, Nat= 10.00, costDye=3880.50, profitDye= 970.12, costNat=3538.50, profitNat= 884.62
   Sweater : Dyed=  3.00, Nat=  2.00, costDye=5880.00, profitDye=1470.00, costNat=5392.00, profitNat=1348.00
   Muffler : Dyed=  3.00, Nat=  2.00, costDye=3233.75, profitDye= 808.44, costNat=2948.75, profitNat= 737.19
   Cap     : Dyed=  3.00, Nat=  2.00, costDye=1217.25, profitDye= 243.45, costNat=1095.25, profitNat= 219.05
   Gloves  : Dyed=  3.00, Nat=  2.00, costDye=1298.50, profitDye= 259.70, costNat=1179.50, profitNat= 235.90
------------------------------------------------

=== SCENARIO: M

In [23]:
import gurobipy as gp
from gurobipy import GRB, quicksum

# =========================================
# 1) Base Data
# =========================================

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw-material capacity (grams)
RM_WOVEN_MAX = 15000  # 15 kg for woven
RM_KNIT_MAX  = 23000  # 23 kg for knit

# Base profit margin (we will add increments to these)
base_profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

# Cost parameters
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25
spin_labour_cost        = 3.00

weave_labour_cost = {
    "Shawl":   1.46,
    "Stole":   1.79,
    "Scarf":   3.00,
    "Sweater": 0.00,
    "Muffler": 0.00,
    "Cap":     0.00,
    "Gloves":  0.00
}

knit_labour_cost = {
    "Shawl":   0.00,
    "Stole":   0.00,
    "Scarf":   0.00,
    "Sweater": 3.00,
    "Muffler": 3.00,
    "Cap":     3.00,
    "Gloves":  3.00
}

finishing_labour_cost = 0.60

dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

# Weavers / knitters
weaver_count  = 5
knitter_count = 13

# Productivity in grams/person/day
weave_prod_g_per_day = 500
knit_prod_g_per_day  = 50

# =========================================
# 2) Cost / Profit Functions
# =========================================

def compute_unit_cost(product, dyed=False):
    """
    Return full cost of producing one unit, including raw, spinning, weaving/knit, finishing, dye if dyed.
    """
    grams = raw_req[product]
    
    # raw material
    if product_type[product] == "woven":
        mat_c = grams * raw_material_cost_woven
    else:
        mat_c = grams * raw_material_cost_knit
    
    # spinning
    spin_c = grams * spin_labour_cost
    
    # weaving or knitting
    wv_c   = grams * weave_labour_cost[product]
    kn_c   = grams * knit_labour_cost[product]
    
    # finishing
    fin_c  = grams * finishing_labour_cost
    
    # dye
    dye_c  = dye_cost[product] if dyed else 0.0
    
    return mat_c + spin_c + wv_c + kn_c + fin_c + dye_c

def compute_unit_profit(product, dyed, margin_dict):
    """
    Profit per unit = cost * margin
    (since Price = cost * (1 + margin)).
    """
    c = compute_unit_cost(product, dyed)
    return c * margin_dict[product]

# =========================================
# 3) Loops Over margin_increments & Days
# =========================================

margin_increments = [0,0.05, 0.10, 0.15, 0.20]
days_range = range(3, 40, 1)  # from 3 to 39 inclusive

for inc in margin_increments:
    # Build scenario margin dictionary
    scenario_margin = {}
    for p in products:
        scenario_margin[p] = base_profit_margin[p] + inc
    
    for D in days_range:
        scenario_label = f"Margin+{int(inc*100)}%_Days={D}"
        
        # Build model
        m = gp.Model(scenario_label)
        m.setParam("OutputFlag", 0)  # quiet solver output
        
        # Decision vars
        xD = {}
        xN = {}
        for p in products:
            xD[p] = m.addVar(vtype=GRB.INTEGER, name=f"xDyed_{p}")
            xN[p] = m.addVar(vtype=GRB.INTEGER, name=f"xNat_{p}")

        # Objective
        m.setObjective(
            quicksum(
                compute_unit_profit(p, True,  scenario_margin)*xD[p] +
                compute_unit_profit(p, False, scenario_margin)*xN[p]
                for p in products
            ),
            GRB.MAXIMIZE
        )

        # Constraints
        
        # (a) Minimum units
        for p in products:
            m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")

        # (b) <= 60% dyed
        for p in products:
            m.addConstr(xD[p] == 0.50*(xD[p] + xN[p]), name=f"DyeLimit_{p}")

        # (c) raw material capacity
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
            <= RM_WOVEN_MAX,
            name="WovenRawCap"
        )
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
            <= RM_KNIT_MAX,
            name="KnitRawCap"
        )

        # (d) Weaving capacity
        weave_cap = weaver_count*weave_prod_g_per_day*D
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
            <= weave_cap,
            "WeaveCap"
        )

        # (e) Knitting capacity
        knit_cap = knitter_count*knit_prod_g_per_day*D
        m.addConstr(
            quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
            <= knit_cap,
            "KnitCap"
        )

        # Solve
        m.optimize()

        # Print results
        print(f"=== SCENARIO: {scenario_label} ===")
        if m.Status == GRB.INFEASIBLE or m.SolCount == 0:
            print("   NO SOLUTION\n")
            continue
        
        print(f"   Optimal Total Profit = {m.ObjVal:.2f}")
        for p in products:
            xd_val = xD[p].X
            xn_val = xN[p].X
            if (xd_val + xn_val) > 1e-6:  # only print details if > 0
                # cost
                cost_d = compute_unit_cost(p, True)
                cost_n = compute_unit_cost(p, False)
                # profit
                prof_d = compute_unit_profit(p, True,  scenario_margin)
                prof_n = compute_unit_profit(p, False, scenario_margin)
                print(f"   {p:8s}: Dyed={xd_val:6.2f}, Nat={xn_val:6.2f}, "
                      f"costDye={cost_d:7.2f}, profitDye={prof_d:7.2f}, "
                      f"costNat={cost_n:7.2f}, profitNat={prof_n:7.2f}")
        print("------------------------------------------------\n")


=== SCENARIO: Margin+0%_Days=3 ===
   NO SOLUTION

=== SCENARIO: Margin+0%_Days=4 ===
   Optimal Total Profit = 52740.47
   Shawl   : Dyed=  1.00, Nat=  1.00, costDye=8638.10, profitDye=2159.53, costNat=7808.10, profitNat=1952.03
   Stole   : Dyed=  5.00, Nat=  5.00, costDye=6734.60, profitDye=1683.65, costNat=6099.60, profitNat=1524.90
   Scarf   : Dyed= 12.00, Nat= 12.00, costDye=3880.50, profitDye= 970.12, costNat=3538.50, profitNat= 884.62
   Sweater : Dyed=  1.00, Nat=  1.00, costDye=5880.00, profitDye=1470.00, costNat=5392.00, profitNat=1348.00
   Muffler : Dyed=  3.00, Nat=  3.00, costDye=3233.75, profitDye= 808.44, costNat=2948.75, profitNat= 737.19
   Cap     : Dyed=  3.00, Nat=  3.00, costDye=1217.25, profitDye= 243.45, costNat=1095.25, profitNat= 219.05
   Gloves  : Dyed=  3.00, Nat=  3.00, costDye=1298.50, profitDye= 259.70, costNat=1179.50, profitNat= 235.90
------------------------------------------------

=== SCENARIO: Margin+0%_Days=5 ===
   Optimal Total Profit = 66960

EXTENSIVE ANALYSIS Without Assumption that spinning, plying, and dyeing are already complete

In [28]:
import gurobipy as gp
from gurobipy import GRB, quicksum

#========================
# 1) DATA SETUP
#========================

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

# Grams of raw material needed per SKU
raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

# Identify which products are woven vs knit
product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

# Minimum units requirement
min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw material availability (in grams): 15 kg for woven, 23 kg for knit
RM_WOVEN_MAX = 15000
RM_KNIT_MAX  = 23000

# Number of workers and days, to compute total capacity
spinner_count = 8
plyer_count   = 8
weaver_count  = 5
knitter_count = 13
D = 49  # days

#========================
# 2) COST PARAMETERS
#========================

# Raw material cost per gram (you can vary these later for sensitivity)
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25

# Labor cost per gram for spinning, weaving, knitting, finishing (from the table)
spin_labour_cost    = 3.00     # spinning is 3.00 INR/g for all
weave_labour_cost   = {
   "Shawl": 1.46,
   "Stole": 1.79,
   "Scarf": 3.00,
   "Sweater": 0.0,    # no weaving for knitted products
   "Muffler": 0.0,
   "Cap": 0.0,
   "Gloves": 0.0
}
knit_labour_cost = {
   "Shawl": 0.0,      # no knitting for woven products
   "Stole": 0.0,
   "Scarf": 0.0,
   "Sweater": 3.00,
   "Muffler": 3.00,
   "Cap": 3.00,
   "Gloves": 3.00
}
finishing_labour_cost = 0.60   # finishing is 0.60 INR/g for all

# Dye cost per SKU (fixed cost per unit if dyed)
dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

# Profit margin for each SKU
profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

#========================
# 3) FUNCTIONS TO COMPUTE COST & PROFIT
#========================

def compute_unit_cost(product, dyed=False):
    """Return total (INR) cost of producing ONE unit of `product` (dyed or not)."""
    grams = raw_req[product]
    
    # 1. Raw material
    if product_type[product] == "woven":
        mat_cost = grams * raw_material_cost_woven
    else:
        mat_cost = grams * raw_material_cost_knit
    
    # 2. Labor
    spin_cost  = grams * spin_labour_cost
    weave_cost = grams * weave_labour_cost[product]
    knit_cost  = grams * knit_labour_cost[product]
    finish_cost= grams * finishing_labour_cost
    
    # 3. Dye cost
    dye_c = dye_cost[product] if dyed else 0.0
    
    return mat_cost + spin_cost + weave_cost + knit_cost + finish_cost + dye_c

def compute_unit_profit(product, dyed=False):
    """Return profit (INR) for producing ONE unit of `product` (dyed or not)."""
    c = compute_unit_cost(product, dyed)
    return c * profit_margin[product]

#========================
# 4) BUILD THE MODEL
#========================

m = gp.Model("Looms_of_Ladakh_Optimization")

# Decision variables: xD[i], xN[i] = # of dyed vs # of natural units
xD = {p: m.addVar(vtype=GRB.INTEGER, name=f"xDyed_{p}", lb=0) for p in products}
xN = {p: m.addVar(vtype=GRB.INTEGER, name=f"xNat_{p}",  lb=0) for p in products}

#========================
# 5) OBJECTIVE: max profit
#========================

m.setObjective(
    quicksum(compute_unit_profit(p, dyed=True)*xD[p] + compute_unit_profit(p, dyed=False)*xN[p]
             for p in products),
    GRB.MAXIMIZE
)

#========================
# 6) CONSTRAINTS
#========================

# (a) Minimum units
for p in products:
    m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")

# (b) Dye limit (<= 60%)
for p in products:
    m.addConstr(xD[p] == 0.60*(xD[p] + xN[p]), name=f"DyeLimit_{p}")

# (c) Raw material constraints
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p] == "woven") <= RM_WOVEN_MAX,
    name="WovenRawLimit"
)
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p] == "knit") <= RM_KNIT_MAX,
    name="KnitRawLimit"
)

# (d) Process capacity constraints
# Productivity: (grams/day/person). Multiply by #persons * days for total capacity.
spin_woven_g_per_day  = 21.5
spin_knit_g_per_day   = 37.5
ply_woven_g_per_day   = 250
ply_knit_g_per_day    = 500
weave_g_per_day       = 500
knit_g_per_day        = 50

total_spinning_days = spinner_count * D
total_plying_days   = plyer_count   * D
total_weaving_days  = weaver_count  * D
total_knitting_days = knitter_count * D

# Spinning capacity: sum of (grams / daily rate) for woven vs knit
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) / spin_woven_g_per_day
             for p in products if product_type[p]=="woven")
  + quicksum(raw_req[p]*(xD[p]+xN[p]) / spin_knit_g_per_day
             for p in products if product_type[p]=="knit")
  <= total_spinning_days,
  name="SpinningCap"
)

# Plying capacity (similarly):
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) / ply_woven_g_per_day
             for p in products if product_type[p]=="woven")
  + quicksum(raw_req[p]*(xD[p]+xN[p]) / ply_knit_g_per_day
             for p in products if product_type[p]=="knit")
  <= total_plying_days,
  name="PlyingCap"
)

# Weaving capacity:
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
  <= weave_g_per_day * total_weaving_days,
  name="WeavingCap"
)

# Knitting capacity:
m.addConstr(
    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
  <= knit_g_per_day * total_knitting_days,
  name="KnittingCap"
)

#========================
# 7) OPTIMIZE
#========================
m.optimize()

#========================
# 8) PRINT SOLUTION
#========================
if m.SolCount > 0:
    print(f"Optimal total profit = {m.ObjVal:.2f} INR")
    for p in products:
        xDyed  = xD[p].X
        xNat   = xN[p].X
        total  = xDyed + xNat
        cost_d = compute_unit_cost(p, dyed=True)
        cost_n = compute_unit_cost(p, dyed=False)
        print(f"{p}: Dyed={xDyed:7.2f}, Nat={xNat:7.2f}, Total={total:7.2f} | "
              f"CostDye={cost_d:7.2f}, CostNat={cost_n:7.2f}")



Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 14 columns and 84 nonzeros
Model fingerprint: 0x4798cdd0
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 5e+02]
  Objective range  [2e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+05]
Presolve removed 20 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 41267.7 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.126766250000e+04, best bound 4.126766250000e+04, gap 0.0000%
Optimal total profit = 41267.66 INR
Shawl: Dyed=   3.00, Nat=   2.00, Total=   5.0

In [27]:
import gurobipy as gp
from gurobipy import GRB, quicksum

# ================================================================
# 1) BASE DATA
# ================================================================

products = ["Shawl","Stole","Scarf","Sweater","Muffler","Cap","Gloves"]

raw_req = {
    "Shawl":   510,
    "Stole":   390,
    "Scarf":   210,
    "Sweater": 320,
    "Muffler": 175,
    "Cap":      65,
    "Gloves":   70
}

product_type = {
    "Shawl":   "woven",
    "Stole":   "woven",
    "Scarf":   "woven",
    "Sweater": "knit",
    "Muffler": "knit",
    "Cap":     "knit",
    "Gloves":  "knit"
}

min_units = {
    "Shawl":   2,
    "Stole":   5,
    "Scarf":  10,
    "Sweater": 2,
    "Muffler": 4,
    "Cap":     5,
    "Gloves":  5
}

# Raw material availability (grams)
RM_WOVEN_MAX = 15000  # 15 kg
RM_KNIT_MAX  = 23000  # 23 kg

# We'll keep weaver_count=5 and knitter_count=13 fixed.
weaver_count  = 5
knitter_count = 13

# Production rates (grams/person/day)
spin_woven_g_per_day  = 21.5
spin_knit_g_per_day   = 37.5
ply_woven_g_per_day   = 250
ply_knit_g_per_day    = 500
weave_g_per_day       = 500
knit_g_per_day        = 50

# ================================================================
# 2) COST PARAMETERS
# ================================================================

# Raw‐material cost per gram
raw_material_cost_woven = 10.25
raw_material_cost_knit  = 10.25

# Labor cost (INR/g)
spin_labour_cost = 3.00

weave_labour_cost = {
   "Shawl":   1.46,
   "Stole":   1.79,
   "Scarf":   3.00,
   "Sweater": 0.00,
   "Muffler": 0.00,
   "Cap":     0.00,
   "Gloves":  0.00
}

knit_labour_cost = {
   "Shawl":   0.00,
   "Stole":   0.00,
   "Scarf":   0.00,
   "Sweater": 3.00,
   "Muffler": 3.00,
   "Cap":     3.00,
   "Gloves":  3.00
}

finishing_labour_cost = 0.60

# Base dye cost (INR per SKU if dyed)
base_dye_cost = {
    "Shawl":   830,
    "Stole":   635,
    "Scarf":   342,
    "Sweater": 488,
    "Muffler": 285,
    "Cap":     122,
    "Gloves":  119
}

# Profit margin for each SKU
profit_margin = {
    "Shawl":   0.25,
    "Stole":   0.25,
    "Scarf":   0.25,
    "Sweater": 0.25,
    "Muffler": 0.25,
    "Cap":     0.20,
    "Gloves":  0.20
}

# ================================================================
# 3) FUNCTIONS FOR COST & PROFIT
# ================================================================

def compute_unit_cost(prod, dyed, dye_cost_dict):
    """Compute the total (INR) cost to produce ONE unit of product 'prod'.
       'dyed' is True/False. 'dye_cost_dict' is the scenario's dye cost data.
    """
    grams = raw_req[prod]
    # Raw material:
    if product_type[prod] == "woven":
        mat_cost = grams * raw_material_cost_woven
    else:
        mat_cost = grams * raw_material_cost_knit
    
    # Labor:
    spin_cost   = grams * spin_labour_cost
    weave_cost  = grams * weave_labour_cost[prod]
    knit_cost   = grams * knit_labour_cost[prod]
    finish_cost = grams * finishing_labour_cost
    
    # Dye cost:
    dye_c = dye_cost_dict[prod] if dyed else 0.0
    
    return mat_cost + spin_cost + weave_cost + knit_cost + finish_cost + dye_c

def compute_unit_profit(prod, dyed, dye_cost_dict):
    """Return the profit (INR) for producing ONE unit of product 'prod' under the given scenario."""
    c = compute_unit_cost(prod, dyed, dye_cost_dict)
    return c * profit_margin[prod]

# ================================================================
# 4) SENSITIVITY SETUP
# ================================================================

# Days in [10,15,20,25,30]
days_list = [10, 15, 20, 25, 30,40,50,60]

# spinner_count & plyer_count in [14..20]
sp_ply_range = range(14, 21)

# Dye cost changes: [-10%, -5%, 0%, +5%, +10%, +15%]
dye_factors = [-0.10, -0.05, 0.0, 0.05, 0.10, 0.15]

# ================================================================
# 5) MAIN NESTED LOOPS
# ================================================================

for D in days_list:
    for sp_count in sp_ply_range:
        for pl_count in sp_ply_range:
            for df in dye_factors:
                
                # Build a scenario-specific dye cost dictionary
                factor = 1.0 + df
                dye_cost_scenario = {}
                for prod in products:
                    dye_cost_scenario[prod] = base_dye_cost[prod] * factor
                
                # Build a new model for this scenario
                scenario_str = (f"[D={D}, spinner={sp_count}, plyer={pl_count}, "
                                f"dyeCostChange={int(df*100)}%]")
                
                m = gp.Model("LoomsOfLadakh_sensitivity")
                m.setParam("OutputFlag", 0)  # reduce Gurobi console output

                # Decision variables
                xD = {p: m.addVar(vtype=GRB.INTEGER, name=f"xD_{p}") for p in products}
                xN = {p: m.addVar(vtype=GRB.INTEGER, name=f"xN_{p}") for p in products}

                # Objective: maximize total profit
                m.setObjective(
                    quicksum(
                        compute_unit_profit(p, True,  dye_cost_scenario)*xD[p] +
                        compute_unit_profit(p, False, dye_cost_scenario)*xN[p]
                        for p in products
                    ),
                    GRB.MAXIMIZE
                )

                # Constraints

                # (a) Minimum units
                for p in products:
                    m.addConstr(xD[p] + xN[p] >= min_units[p], name=f"MinUnits_{p}")

                # (b) At most 60% dyed
                for p in products:
                    m.addConstr(xD[p] == 0.60*(xD[p] + xN[p]), name=f"DyeLimit_{p}")

                # (c) Raw material
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p])
                             for p in products if product_type[p]=="woven")
                    <= RM_WOVEN_MAX,
                    name="RM_woven"
                )
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p])
                             for p in products if product_type[p]=="knit")
                    <= RM_KNIT_MAX,
                    name="RM_knit"
                )

                # (d) Process capacity
                # total capacity = #workers * days
                total_spinning_days = sp_count * D
                total_plying_days   = pl_count * D
                total_weaving_days  = weaver_count  * D
                total_knitting_days = knitter_count * D

                # Spinning
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p]) / spin_woven_g_per_day
                             for p in products if product_type[p]=="woven")
                  + quicksum(raw_req[p]*(xD[p]+xN[p]) / spin_knit_g_per_day
                             for p in products if product_type[p]=="knit")
                  <= total_spinning_days,
                  name="SpinCap"
                )

                # Plying
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p]) / ply_woven_g_per_day
                             for p in products if product_type[p]=="woven")
                  + quicksum(raw_req[p]*(xD[p]+xN[p]) / ply_knit_g_per_day
                             for p in products if product_type[p]=="knit")
                  <= total_plying_days,
                  name="PlyCap"
                )

                # Weaving
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="woven")
                  <= weave_g_per_day * total_weaving_days,
                  name="WeaveCap"
                )

                # Knitting
                m.addConstr(
                    quicksum(raw_req[p]*(xD[p]+xN[p]) for p in products if product_type[p]=="knit")
                  <= knit_g_per_day * total_knitting_days,
                  name="KnitCap"
                )

                # Solve
                m.optimize()

                # Check feasibility
                if m.Status == GRB.INFEASIBLE or m.SolCount == 0:
                    print(f"{scenario_str} --> NO SOLUTION")
                else:
                    # Print total objective + breakdown by product
                    print(f"{scenario_str} --> Profit={m.ObjVal:0.2f}")
                    for p in products:
                        xd_val = xD[p].X
                        xn_val = xN[p].X
                        # compute profit from each product:
                        profit_p = (
                            compute_unit_profit(p, True,  dye_cost_scenario)*xd_val +
                            compute_unit_profit(p, False, dye_cost_scenario)*xn_val
                        )
                        print(f"    {p:8s} Dyed={xd_val:6.2f}, Natural={xn_val:6.2f}, "
                              f"Profit={profit_p:8.2f}")
                    print("------------------------------------------------")


[D=10, spinner=14, plyer=14, dyeCostChange=-10%] --> NO SOLUTION
[D=10, spinner=14, plyer=14, dyeCostChange=-5%] --> NO SOLUTION
[D=10, spinner=14, plyer=14, dyeCostChange=0%] --> NO SOLUTION
[D=10, spinner=14, plyer=14, dyeCostChange=5%] --> NO SOLUTION
[D=10, spinner=14, plyer=14, dyeCostChange=10%] --> NO SOLUTION
[D=10, spinner=14, plyer=14, dyeCostChange=15%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=-10%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=-5%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=0%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=5%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=10%] --> NO SOLUTION
[D=10, spinner=14, plyer=15, dyeCostChange=15%] --> NO SOLUTION
[D=10, spinner=14, plyer=16, dyeCostChange=-10%] --> NO SOLUTION
[D=10, spinner=14, plyer=16, dyeCostChange=-5%] --> NO SOLUTION
[D=10, spinner=14, plyer=16, dyeCostChange=0%] --> NO SOLUTION
[D=10, spinner=14, plyer=16, dyeCostChange